In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedGroupKFold,StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
pd.options.display.max_rows = 999
DATA_PATH = '../Data/Data desafío BCI Challenge/'
FOLDS = 5
SEED  = 42

In [2]:
%%time
train_df = pd.read_csv(os.path.join(DATA_PATH,'train_data.csv'))
test_df  = pd.read_csv(os.path.join(DATA_PATH,'test_data.csv'))

CPU times: user 3 s, sys: 497 ms, total: 3.5 s
Wall time: 3.5 s


# Generated Datasets

## 1.Binning the Target Mes:

In [6]:
n_bins = 7
train_df['target_bin'] = train_df.groupby(['mes'])['target_mes'].transform(lambda x: pd.qcut(x, n_bins))
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_df['target_bin'] = le.fit_transform(train_df['target_bin'])

# New Clients Model:
+ In this case clients that appear in train, does not appear in test
    + Months: [201802-202104]
+ Apply groupkfold on id for predicting in new clients
    + Use train data from 201802 - 202104**

In [7]:
skf = StratifiedGroupKFold(n_splits = FOLDS,shuffle = True, random_state = SEED)
df_fold = train_df[train_df['mes']<=202104].reset_index(drop = True)
for fold,(train_index, test_index) in enumerate(skf.split(df_fold,df_fold['target_bin'],df_fold['id'])):
    df_fold.loc[test_index,'fold'] = fold
df_fold.drop(columns = ['target_bin']).to_parquet(os.path.join("../Data/DatasetsGenerated",'folds_new_clients_model.parquet.gzip'),index = False,compression = 'gzip')

In [8]:
df_fold.groupby(['fold']).target_mes.mean().reset_index()

,fold,target_mes
0,0.0,183.351777
1,1.0,194.849639
2,2.0,184.686757
3,3.0,180.038005
4,4.0,177.548542


## Behavior Model & Future companies with no previous target
+ Stratified Kfold: ['target_bin']
+ Predict sales for clients which you already have information
    + Test months : [202105-202109]
+ For Stratified you can use mes and target_bin
    + Use train data from 201802 - 202109**
**Another idea could be a time split validation[check the tendency for those clients]** 

**You could optimize only in the clients that are on the test**

In [11]:
skf = StratifiedKFold(n_splits = FOLDS,shuffle = True, random_state = SEED)
df_fold = train_df.copy()
for fold,(train_index, test_index) in enumerate(skf.split(df_fold,df_fold['target_bin'])):
    df_fold.loc[test_index,'fold'] = fold
df_fold.drop(columns = ['target_bin']).to_parquet(os.path.join("../Data/DatasetsGenerated",'folds_behavior_clients_model.parquet.gzip'),index = False,compression = 'gzip')